# Empirical Bernstein Stopping (EBS) Algorithm

The empirical Bernstein stopping (EBS) algorithm is a stopping algorithm which determines when to stop a sampling process.
EBS is build on the empirical Bernstein inequality [[1](https://www.cs.toronto.edu/~vmnih/docs/ebstop.pdf)], which is a modification of the Bernstein inequality.

The EBS algorithm depends on the following information:
- the range $R$ of the random variables --> $\forall X_i: a_i \leq X_i \leq b_i$ hence $R = b - a$,
- the empirical variance $\overline{\sigma^2}_t=\frac{1}{t}\sum^t_{i=1}(X_i-\overline{X}_t)^2$, with $t$ being the amount of samples taken,

of the random variable $X$.

This algorithm returns an $(\epsilon,\delta)$-estimate which is at most $\epsilon$ from its actual mean, $1-\delta$ of the times.\
Furthermore, it the following parameters characterize EBS:
- the accuracy $\epsilon$, which can be absolute or relative,
- the confidence $1-\delta$, which is the probability to return an $(\epsilon,\delta)$-estimate.

In addition to various modifications to EBS, algorithms which are based on the Höffding's inequality are also included.\
This notebook may serve as a demonstration on how to use the provided algorithms.

## Example

In [3]:
import sys
import numpy as np
import matplotlib.pyplot as plt

# Importing the algorithms
sys.path.append("./")
import src.algorithms as alg

Define some variables for the random variable/ distribution and EBS.\
In this example, a uniform distribution is used.

In [4]:
a = 0
b = 1
R = b - a 
delta = 0.1
epsilon = 0.01

Create an instance of the `eba_geo_marg` ebs class.

In [5]:
ebs = alg.eba_geo_marg(epsilon=epsilon,delta=delta,range_of_rndvar= R)

The loop where the algorithm is actually running.

In [6]:
while ebs.cond_check():
    ebs.add_sample(np.random.uniform(a,b))
    if ebs.inner_cond_check():
        ebs.update_ct()

When terminated, various one can return various information from EBS.

In [8]:
print('Estimate: ',ebs.get_estimate())
print('# Samples:',ebs.get_step())

Estimate:  0.501474731190747
# Samples: 2254


***

The other algorithms behave exactly the same. For more information about the modifications one can take my bachelor [thesis](pdf/Bachelor_Thesis.pdf) or the paper of Mnih et. al. [[1](https://www.cs.toronto.edu/~vmnih/docs/ebstop.pdf)] to hand.